In [5]:
import numpy as np
import pandas as pd
import pydicom
import glob, os
import pydicom
from pydicom.filebase import DicomBytesIO
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

from nvidia.dali import pipeline_def
import nvidia.dali.fn as fn
import nvidia.dali.types as types
from nvidia.dali.types import DALIDataType

In [7]:
class ExternalInputIterator(object):
    def __init__(self,jpeg_stream):
        self.jpeg_stream = jpeg_stream
    def __iter__(self):
        return self
    def __next__(self):
        return self.jpeg_stream

    
@pipeline_def
def j2k_decode_pipeline(eii, is_resize=False, image_size = None):
    jpeg  = dali.fn.external_source(source=eii, dtype=dali.types.UINT8)
    image = dali.fn.experimental.decoders.image(jpeg, device='mixed', output_type=dali.types.ANY_DATA, dtype=dali.types.UINT16)
    if is_resize:
        image = dali.fn.resize(image, size=image_size)
    return image


In [ ]:
def make_transfer_syntax_uid(df):
    machine_id_to_transfer = {}
    machine_id = df.machine_id.unique()
    for i in machine_id:
        d = df[df.machine_id==i].iloc[0]
        f = f'{dcm_dir}/{d.patient_id}/{d.image_id}.dcm'
        dicom = pydicom.dcmread(f)
        machine_id_to_transfer[i]= dicom.file_meta.TransferSyntaxUID
    return machine_id_to_transfer